In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip '/content/drive/MyDrive/TCC LOCAL/projeto/datasets/fer.zip' >> /dev/null

replace __MACOSX/._fer? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/fer/._test? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Primeira Etapa: Extração de caracteristicas

In [3]:
import os
from os.path import join as join_path
import cv2
import json
import tensorflow as tf
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np

SEED = 42

def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

BASE_PATH = join_path('/content', 'fer')
BASE_CSV_PATH = '/content/drive/MyDrive/TCC LOCAL/projeto/outputs_treinamento/vgg16_svm'
BASE_CSV_PATH = join_path('/content', 'outputs_treinamento')
BATCH_SIZE = 32
LE_PATH = BASE_CSV_PATH + '/le.csv'
MODEL_PATH = '/content/drive/MyDrive/TCC LOCAL/projeto/outputs_treinamento/vgg16_svm/model.sav'

"""
images_folder = join_path('/content', 'fer')
img_size = 224

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  join_path(images_folder, 'train'),
  image_size=(img_size, img_size),
  seed=SEED
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  join_path(images_folder, 'test'),
  image_size=(img_size, img_size),
  seed=SEED
)
"""

"\nimages_folder = join_path('/content', 'fer')\nimg_size = 224\n\ntrain_dataset = tf.keras.preprocessing.image_dataset_from_directory(\n  join_path(images_folder, 'train'),\n  image_size=(img_size, img_size),\n  seed=SEED\n)\n\nvalidation_dataset = tf.keras.preprocessing.image_dataset_from_directory(\n  join_path(images_folder, 'test'),\n  image_size=(img_size, img_size),\n  seed=SEED\n)\n"

In [4]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from imutils import paths
import numpy as np
import pickle
import random
import os
# load the VGG16 network and initialize the label encoder
print("[INFO] loading network...")
model = VGG16(weights="imagenet", include_top=False)
le = None

[INFO] loading network...


In [5]:
for split in ('train', 'test'):
  print("[INFO] processing '{} split'...".format(split))
  p = os.path.sep.join([BASE_PATH, split])
  imagePaths = list(paths.list_images(p))
  # randomly shuffle the image paths and then extract the class
  # labels from the file paths
  random.shuffle(imagePaths)
  labels = [p.split(os.path.sep)[-2] for p in imagePaths]
  # if the label encoder is None, create it
  if le is None:
    le = LabelEncoder()
    le.fit(labels)

  # open the output CSV file for writing
  csvPath = os.path.sep.join([BASE_CSV_PATH,
    "{}.csv".format(split)])
  with open(csvPath, "w") as csv:
    # loop over the images in batches
    for (b, i) in enumerate(range(0, len(imagePaths), BATCH_SIZE)):
      # extract the batch of images and labels, then initialize the
      # list of actual images that will be passed through the network
      # for feature extraction
      print("[INFO] processing batch {}/{}".format(b + 1,
        int(np.ceil(len(imagePaths) / float(BATCH_SIZE)))))
      batchPaths = imagePaths[i:i + BATCH_SIZE]
      batchLabels = le.transform(labels[i:i + BATCH_SIZE])
      batchImages = []
    
      for imagePath in batchPaths:
        # load the input image using the Keras helper utility
        # while ensuring the image is resized to 224x224 pixels
        image = load_img(imagePath, target_size=(224, 224))
        image = img_to_array(image)
        # preprocess the image by (1) expanding the dimensions and
        # (2) subtracting the mean RGB pixel intensity from the
        # ImageNet dataset
        image = np.expand_dims(image, axis=0)
        image = preprocess_input(image)
        # add the image to the batch
        batchImages.append(image)
        
      batchImages = np.vstack(batchImages)
      features = model.predict(batchImages, batch_size=BATCH_SIZE)
      features = features.reshape((features.shape[0], 7 * 7 * 512))
  
      # loop over the class labels and extracted features
      for (label, vec) in zip(batchLabels, features):
        # construct a row that exists of the class label and
        # extracted features
        vec = ",".join([str(v) for v in vec])
        csv.write("{},{}\n".format(label, vec))

# serialize the label encoder to disk
f = open(LE_PATH, "wb")
f.write(pickle.dumps(le))
f.close()

[INFO] processing 'train split'...
[INFO] processing batch 1/898
[INFO] processing batch 2/898
[INFO] processing batch 3/898
[INFO] processing batch 4/898
[INFO] processing batch 5/898
[INFO] processing batch 6/898
[INFO] processing batch 7/898
[INFO] processing batch 8/898
[INFO] processing batch 9/898
[INFO] processing batch 10/898
[INFO] processing batch 11/898
[INFO] processing batch 12/898
[INFO] processing batch 13/898
[INFO] processing batch 14/898
[INFO] processing batch 15/898
[INFO] processing batch 16/898
[INFO] processing batch 17/898
[INFO] processing batch 18/898
[INFO] processing batch 19/898
[INFO] processing batch 20/898
[INFO] processing batch 21/898
[INFO] processing batch 22/898
[INFO] processing batch 23/898
[INFO] processing batch 24/898
[INFO] processing batch 25/898
[INFO] processing batch 26/898
[INFO] processing batch 27/898
[INFO] processing batch 28/898
[INFO] processing batch 29/898
[INFO] processing batch 30/898
[INFO] processing batch 31/898
[INFO] proces

# Fase de treinamento

In [5]:
# import the necessary packages
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
import pickle
import os
def load_data_split(splitPath):
	# initialize the data and labels
	data = []
	labels = []
	# loop over the rows in the data split file
	for row in open(splitPath):
		# extract the class label and features from the row
		row = row.strip().split(",")
		label = row[0]
		features = np.array(row[1:], dtype="float")
		# update the data and label lists
		data.append(features)
		labels.append(label)
	# convert the data and labels to NumPy arrays
	data = np.array(data)
	labels = np.array(labels)
	# return a tuple of the data and labels
	return (data, labels)

In [6]:
# derive the paths to the training and testing CSV files
trainingPath = os.path.sep.join([BASE_CSV_PATH,
	"{}.csv".format('train')])
testingPath = os.path.sep.join([BASE_CSV_PATH,
	"{}.csv".format('test')])
# load the data from disk
print("[INFO] loading data...")
(trainX, trainY) = load_data_split(trainingPath)
(testX, testY) = load_data_split(testingPath)
# load the label encoder from disk
le = pickle.loads(open(LE_PATH, "rb").read())

[INFO] loading data...


In [ ]:
# train the model
print("[INFO] training model...")
""" ORIGINAL
model = LogisticRegression(solver="lbfgs", multi_class="auto",
	max_iter=150)
"""
model = LogisticRegression(solver="lbfgs", multi_class="auto",
	max_iter=150)
model.fit(trainX, trainY)
# evaluate the model
print("[INFO] evaluating...")
preds = model.predict(testX)
print(classification_report(testY, preds, target_names=le.classes_))


[INFO] training model...


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[INFO] evaluating...
              precision    recall  f1-score   support

       angry       0.40      0.38      0.39       958
     disgust       0.53      0.49      0.51       111
        fear       0.35      0.38      0.37      1024
       happy       0.70      0.71      0.70      1774
     neutral       0.45      0.44      0.45      1233
         sad       0.38      0.37      0.38      1247
    surprise       0.67      0.68      0.67       831

    accuracy                           0.51      7178
   macro avg       0.50      0.49      0.49      7178
weighted avg       0.51      0.51      0.51      7178



In [7]:
# train the model
print("[INFO] training model...")
model = LogisticRegression(solver="lbfgs", multi_class="auto", penalty='l2', C=0.001,
	max_iter=150)
model.fit(trainX, trainY)
# evaluate the model
print("[INFO] evaluating...")
preds = model.predict(testX)
print(classification_report(testY, preds, target_names=le.classes_))

[INFO] training model...


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[INFO] evaluating...
              precision    recall  f1-score   support

       angry       0.41      0.39      0.40       958
     disgust       0.57      0.46      0.51       111
        fear       0.37      0.37      0.37      1024
       happy       0.69      0.73      0.71      1774
     neutral       0.46      0.46      0.46      1233
         sad       0.39      0.38      0.38      1247
    surprise       0.69      0.69      0.69       831

    accuracy                           0.52      7178
   macro avg       0.51      0.50      0.50      7178
weighted avg       0.51      0.52      0.52      7178



In [9]:
# serialize the model to disk
print("[INFO] saving model...")
f = open(MODEL_PATH, "wb")
f.write(pickle.dumps(model))
f.close()

[INFO] saving model...


In [10]:
# 2nd chance without limit
print("[INFO] training model...")
model = LogisticRegression(solver="lbfgs", multi_class="auto", penalty='l2', C=0.001)
model.fit(trainX, trainY)
# evaluate the model
print("[INFO] evaluating...")
preds = model.predict(testX)
print(classification_report(testY, preds, target_names=le.classes_))

[INFO] training model...


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[INFO] evaluating...
              precision    recall  f1-score   support

       angry       0.42      0.42      0.42       958
     disgust       0.52      0.47      0.49       111
        fear       0.38      0.38      0.38      1024
       happy       0.70      0.74      0.72      1774
     neutral       0.48      0.48      0.48      1233
         sad       0.40      0.38      0.39      1247
    surprise       0.69      0.68      0.68       831

    accuracy                           0.53      7178
   macro avg       0.51      0.51      0.51      7178
weighted avg       0.52      0.53      0.52      7178



In [11]:
# serialize the model to disk
print("[INFO] saving model...")
f = open(MODEL_PATH, "wb")
f.write(pickle.dumps(model))
f.close()

[INFO] saving model...
